<a href="https://colab.research.google.com/github/ghitabnr/Orange-detector-model/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch
from pathlib import Path
import torch
import os
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -r requirements.txt

In [ ]:
model_path = '/content/drive/MyDrive/ColabNotebooks/best.pt'
model = torch.load(model_path, map_location=torch.device('cpu'))

In [ ]:
!python /content/yolov5/detect.py --weights /content/drive/MyDrive/ColabNotebooks/best.pt --img 672 --conf 0.1 --source /content/drive/MyDrive/ColabNotebooks/data_combined/dataset1/OrangeDetection1/valid/images

In [ ]:
!python /content/yolov5/detect.py --weights /content/drive/MyDrive/ColabNotebooks/best.pt --img 672 --conf 0.1 --source /content/drive/MyDrive/ColabNotebooks/data_combined/dataset2/cropload/valid/images

In [ ]:
!python /content/yolov5/detect.py --weights /content/drive/MyDrive/ColabNotebooks/best.pt --img 672 --conf 0.1 --source /content/drive/MyDrive/ColabNotebooks/data_combined/dataset3/FinalDataset/valid/images

In [ ]:
!python /content/yolov5/detect.py --weights /content/drive/MyDrive/ColabNotebooks/best.pt --img 672 --conf 0.1 --source /content/drive/MyDrive/ColabNotebooks/test/video

In [ ]:
import os
from PIL import Image

# Function to crop an image into 4 pieces
def crop_image(image):
    width, height = image.size
    piece_width = width // 2
    piece_height = height // 2
    pieces = []
    for j in range(2):
        for i in range(2):
            box = (i * piece_width, j * piece_height, (i + 1) * piece_width, (j + 1) * piece_height)
            piece = image.crop(box)
            pieces.append(piece)
    return pieces

# Path to the directory containing the input images
input_directory = "/content/drive/MyDrive/ColabNotebooks/test/imagesinput"
output_directory = "/content/drive/MyDrive/ColabNotebooks/test/imagesdecoupees"

# Iterate through all images in the directory
for filename in os.listdir(input_directory):
    if filename.endswith(".jpg") or filename.endswith(".png") or filename.endswith(".JPG"): # Filter image files
        # Load the image
        image_path = os.path.join(input_directory, filename)
        image = Image.open(image_path)

        # Crop the image into pieces
        pieces = crop_image(image)

        # Save the pieces in the output directory
        for idx, piece in enumerate(pieces):
            piece_filename = f"{filename.split('.')[0]}%piece{idx}.JPG"
            piece_path = os.path.join(output_directory, piece_filename)
            piece.save(piece_path)

            # Display the positions of each piece
            print(f"Position of {piece_filename}: {idx % 2}, {idx // 2}")

print("Cropping completed.")


In [ ]:
!python /content/yolov5/detect.py  --weights /content/drive/MyDrive/ColabNotebooks/best.pt  --img 672 --conf 0.1 --source /content/drive/MyDrive/ColabNotebooks/test/imagesdecoupees

In [ ]:
import os
from PIL import Image

# Function to reconstruct an image from pieces
def reconstruct_image(pieces, output_path):
    # Get dimensions of a piece
    piece_width, piece_height = pieces[0].size

    # Calculate dimensions of the reconstructed image
    image_width = piece_width * 2
    image_height = piece_height * 2

    # Create a new image to reconstruct the original image
    reconstructed_image = Image.new("RGB", (image_width, image_height))

    # Assemble the pieces to reconstruct the original image
    for idx, piece in enumerate(pieces):
        x = idx % 2
        y = idx // 2
        reconstructed_image.paste(piece, (x * piece_width, y * piece_height))

    # Save the reconstructed image
    reconstructed_image.save(output_path)

# Path to the directory containing image pieces
input_directory = "/content/yolov5/runs/detect/exp"
output_directory = "/content/drive/MyDrive/ColabNotebooks/test/imagesoutput"

# Iterate through all image pieces in the directory
image_pieces = {}
for filename in sorted(os.listdir(input_directory)):  # Added sorted() here
    if filename.endswith(".jpg") or filename.endswith(".png") or filename.endswith(".JPG"):  # Filter image files
        # Extract the name of the original image from the piece name
        image_name = filename.split("%")[0]

        # Load the image piece
        piece_path = os.path.join(input_directory, filename)
        piece = Image.open(piece_path)

        # Add the piece to the list of pieces associated with the original image
        if image_name not in image_pieces:
            image_pieces[image_name] = []
        image_pieces[image_name].append(piece)

# Before reconstituting each image, sort the pieces by their intended order
for image_name, pieces in image_pieces.items():
    # Assuming your pieces are named with an ending number indicating their order
    pieces_sorted = sorted(pieces, key=lambda x: int(x.filename.split('%')[-1].split('.')[0].replace('piece', '')))
    # Output path for the reconstructed image
    output_path = os.path.join(output_directory, f"{image_name}%reconstructed.JPG")
    # Reconstruct the image with the sorted pieces and save
    reconstruct_image(pieces_sorted, output_path)

print("Reconstruction completed.")


In [ ]:
import os
import shutil

def empty_directory(directory):
    """
    Vide un répertoire en supprimant tous ses fichiers et sous-répertoires.
    Args:
        directory (str): Chemin du répertoire à vider.
    """
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Erreur lors de la suppression du fichier %s : %s' % (file_path, e))

# Spécifiez les chemins des répertoires que vous souhaitez vider
directories_to_empty = [
    '/content/yolov5/runs/detect',
    '/content/drive/MyDrive/ColabNotebooks/test/video',
    '/content/drive/MyDrive/ColabNotebooks/test/imagesoutput',
    '/content/drive/MyDrive/ColabNotebooks/test/imagesinput',
    '/content/drive/MyDrive/ColabNotebooks/test/imagesdecoupees',

    # Ajoutez d'autres répertoires au besoin
]

# Videz chaque répertoire spécifié
for directory in directories_to_empty:
    print("Vidage du répertoire :", directory)
    empty_directory(directory)
